In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC

import xgboost
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import time
import numpy as np
import datetime
from os.path import exists

from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
data = np.loadtxt('data/latent_fpkm_gistic_rppa.csv', delimiter=',')
print(data.shape)

labels = np.loadtxt('data/encoded_labels.csv')
print(labels.shape)


now = datetime.datetime.now()
Time_Now= now.strftime("%Y-%m-%d %H:%M:%S")

PC= 'Lenovo'
dataset= 'Pan'
target_names=['LumA','LumB', 'Basal', 'Her2',]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

num_classes = len(np.unique(labels))
num_input=data.shape[1]



(511, 100)
(511,)


In [3]:
#produce results
path = 'data/results_fpkm_gistic_rppa/'

def write_results(results, path):
    
    file_exists = exists(path)
    if not(file_exists):
        with open(path,"a+") as f:
            f.write(results)
            f.write("\n")
    else:
        with open(path,"a+") as f:
            f.write(results)

In [4]:
dtc =  DecisionTreeClassifier()
rfc = RandomForestClassifier()
knn =  KNeighborsClassifier()
xgb = xgboost.XGBClassifier()
svm = SVC(probability=True)

In [5]:
'''clf = [dtc,rfc,knn,xgb]
for algo in clf:
    score = cross_val_score( algo,X,y,cv = 10,scoring = 'accuracy')
    print("The accuracy score of {} is:".format(algo),score.mean())'''

'clf = [dtc,rfc,knn,xgb]\nfor algo in clf:\n    score = cross_val_score( algo,X,y,cv = 10,scoring = \'accuracy\')\n    print("The accuracy score of {} is:".format(algo),score.mean())'

In [6]:
dtc =  DecisionTreeClassifier()
rfc = RandomForestClassifier()
knn =  KNeighborsClassifier()
xgb = xgboost.XGBClassifier()
svm = SVC(probability=True)
classifiers = [('dtc',dtc),('rfc',rfc),('knn',knn) ,('xgb',xgb)] #, ('svm',svm)list of (str, estimator)

In [7]:
t1= time.time()
lr = LogisticRegression()
clf = StackingClassifier( estimators = classifiers,final_estimator = lr)


clf.fit(X_train , y_train)
#score = cross_val_score(stack_model,new_X,y,cv = 10,scoring = 'accuracy')
#score
y_predict=clf.predict(X_test)
acc=accuracy_score(y_predict,y_test)
prc=precision_score(y_predict,y_test, average='weighted')
rec=recall_score(y_predict,y_test, average='weighted')
f1=f1_score(y_predict,y_test, average='weighted')

results ='Acc: %.2f' % (acc*100) + ', Pre: %.2f' % (prc*100)
results += ', Rec: %.2f' % (rec*100) + ', F1: %.2f' % (f1*100) + ', PC:Lenovo'
results += 'Dataset: ' + dataset + '\n'
    



print (results) 
t2=time.time()
t2-t1

Acc: 82.52, Pre: 83.40, Rec: 82.52, F1: 82.61, PC:LenovoDataset: Pan



3.805452585220337

In [8]:
results+= 'Done in time: ' + str(round(t2-t1))

path2= path + "Ensemble_ML_results.txt" # the name of the file
write_results(results, path2)
